# HVAC / Auxiliary Energy Budget (Simple Engineering Model)

This notebook estimates auxiliary energy consumption (e.g., HVAC load) and its impact on EV range.

**Goal:** Provide a simple, explainable baseline model with transparent assumptions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def energy_kwh(power_kw: float, hours: float) -> float:
    return power_kw * hours

def range_impact_km(battery_kwh: float, consumption_kwh_per_km: float, aux_kw: float, trip_hours: float) -> dict:
    base_range_km = battery_kwh / consumption_kwh_per_km
    aux_kwh = energy_kwh(aux_kw, trip_hours)
    remaining_kwh = max(battery_kwh - aux_kwh, 0)
    range_with_aux_km = remaining_kwh / consumption_kwh_per_km
    return {
        "base_range_km": base_range_km,
        "aux_kwh": aux_kwh,
        "range_with_aux_km": range_with_aux_km,
        "range_loss_km": base_range_km - range_with_aux_km
    }

In [ ]:
battery_kwh = 75
consumption_kwh_per_km = 0.18
trip_hours = 2.0

aux_levels_kw = [0.5, 1.0, 2.0, 3.0, 5.0]
rows = []
for aux in aux_levels_kw:
    out = range_impact_km(battery_kwh, consumption_kwh_per_km, aux, trip_hours)
    out["aux_kw"] = aux
    rows.append(out)

df = pd.DataFrame(rows)[["aux_kw","aux_kwh","base_range_km","range_with_aux_km","range_loss_km"]]
df

In [ ]:
plt.figure()
plt.plot(df["aux_kw"], df["range_loss_km"], marker="o")
plt.xlabel("Auxiliary Load (kW)")
plt.ylabel("Range Loss over 2 hours (km)")
plt.title("Estimated Range Loss vs Auxiliary Load")
plt.grid(True)
plt.show()

## Interpretation (Engineering Notes)

- Auxiliary loads consume energy based on **time**, not distance.
- At low-speed or stop-and-go driving, the effective range penalty becomes more significant.
- This is a simplified baseline model intended for quick energy budgeting and early concept trade-offs.